In [6]:
from RSA import RSA
import numpy as np
import benchmark_functions as bf
from mealpy.swarm_based.PSO import OriginalPSO
from mealpy.math_based.SCA import BaseSCA
from mealpy.evolutionary_based.GA import BaseGA
from tqdm import tqdm
import pandas as pd
import pickle

In [21]:
# PARAMETROS COMUNES

epochs = 1000
pop_size = 100
dims = 5

funciones = [
  bf.Schwefel(n_dimensions=dims),
  bf.Ackley(n_dimensions=dims),
  bf.Michalewicz(n_dimensions=dims),
  bf.Rastrigin(n_dimensions=dims),
  bf.Rosenbrock(n_dimensions=dims),
  bf.DeJong3(n_dimensions=dims),
  bf.StyblinskiTang(n_dimensions=dims),
  bf.EggHolder(n_dimensions=dims),
  bf.Rana(n_dimensions=dims),
]

algorithms = [
    RSA,
    OriginalPSO,
    BaseSCA,
    BaseGA
]

In [18]:
try:
    with open('tuning.pickle','rb') as f:
        best_params = pickle.load(f)
    print(best_params)
except:
    print('No hay archivo tunig')

               Funcion  alpha   beta
0             Schwefel  0.050  0.005
1               Ackley  0.005  0.001
2          Michalewicz  0.100  0.050
3            Rastrigin  0.005  0.001
4           Rosenbrock  0.100  0.001
5            De Jong 3  0.005  0.500
6  Styblinski and Tang  0.050  0.010
7           Egg Holder  0.010  0.100
8                 Rana  0.010  0.100


In [22]:
resultados_min = []
resultados_mean = []

for func in tqdm(funciones):
  fila_min = [func.name()]
  fila_mean = [func.name()]

  LB,UB = func.suggested_bounds()
  problem = {
      "fit_func": func,
      "lb": LB,
      "ub": UB,
      "minmax": "min",
      "log_to": None,
      "save_population": False,
      "verbose": False
  }

  # Parametros tuneados
  if type(best_params) == pd.core.frame.DataFrame:
    alpha = best_params[best_params['Funcion']==func.name()]['alpha'].values[0]
    beta = best_params[best_params['Funcion']==func.name()]['beta'].values[0]
  else:
    alpha = 0.1
    beta = 0.1

  for optimizer in algorithms:
    r = []
    for i in range(10):
      if model.name=='RSA':
        model = optimizer(epochs,pop_size,alpha=alpha,beta=beta)
      else:
        model = optimizer(epochs,pop_size)
      
      best_position, best_fitness = model.solve(problem)
      r.append(best_fitness)
    fila_min.append(np.min(r))
    fila_mean.append(np.mean(r))

  # FINAL
  resultados_min.append(fila_min)
  resultados_mean.append(fila_mean)


100%|██████████| 9/9 [00:25<00:00,  2.89s/it]


In [23]:
df_mean = pd.DataFrame(data=resultados_mean,columns=['Function','RSA','PSO','SCA','GA'])
df_mean

,Function,RSA,PSO,SCA,GA
0,Schwefel,806.347013,899.994358,903.283033,212.008790
1,Ackley,0.001207,13.757842,11.528075,8.409196
2,Michalewicz,-3.330299,-2.510940,-2.539250,-4.349953
3,Rastrigin,0.000011,27.823264,26.383153,7.418809
4,Rosenbrock,3.901688,34.265117,30.098951,13.058309
5,De Jong 3,-19.000000,-20.000000,-18.600000,-20.000000
6,Styblinski and Tang,-171.567871,-158.475824,-158.637384,-189.810084
7,Egg Holder,-1798.273049,-2044.559883,-2069.359743,-2553.304702
8,Rana,-1158.180267,-1679.743056,-1531.899998,-1567.621595


In [24]:
cols = ['RSA','PSO','SCA','GA']
min = df_mean[cols].min(axis=1)
df2 = pd.DataFrame()
for col in cols:
    df2[col] = df_mean[col] == min
df2.sum()


RSA    3
PSO    2
SCA    0
GA     5
dtype: int64